## Manipulando datos categóricos

Comúnmente los conjuntos de datos conteinen datos categóricos. Es importante distinguir entre datos categóricos __ordinales__ y __nominales__.

- __Ordinales__: Son aquellos que pueden ser ordenados (p.e., en la talla de una prenda $M < G < XG$).

- __Nominales__: No tiene mucho sentido pensar en un orden(p,e, en el color de una prenda).


In [2]:
import pandas as pd
import numpy as np

In [36]:

df = pd . DataFrame ([ 
    ['verde', 'M' , 10.3 , 'clase1'],
    ['rojo', 'G' , 14.2 , 'clase2'],
    ['azul', 'XG' , 15.6 , 'clase1'] ])

df.columns = ['color', 'talla' , 'precio', 'clase']
df


,color,talla,precio,clase
0,verde,M,10.3,clase1
1,rojo,G,14.2,clase2
2,azul,XG,15.6,clase1


Este conjunto de datos contiene una característica _nominal_(color) una _ordinal_(talla) y una _numérica_(precio). Este conjunto puede utilizarse en un __algortimo supervisado__, dado que las etiquetas de la clase a la que pertenece cada muestra en la última columna.

## Mapeando valores ordinales

Para garantizar que nuestros algoritmos interpreten correctamente características ordinales, es necesario convertir estas características en valores enteros; desafortunadamente no hay una función que pueda determinar automáticamente el orden correcto a partir de las etiquetas de la talla; por lo tanto, en general se debe realizar este proceso manualmente (con ayuda de un diccionario):

In [37]:
talla_map = {'XG': 3, 'G': 2, 'M': 1}
df.talla = df['talla'].map(talla_map)
df

,color,talla,precio,clase
0,verde,1,10.3,clase1
1,rojo,2,14.2,clase2
2,azul,3,15.6,clase1


## Codificando las etiquetas
Muchas algortirmos de aprendizaje requieren que las __etiquetas de clase esten codificados como números enteros__. Aunque la mayoría de los algoritmos que provee _sckit-learn_ pueden realizar esta conversión internamente, es una buena práctica proveer las etiquetas como enteros y como las clases no son ordinales no importa que número se asigna a cada una de ellas. Por lo tanto, podemos enumerar las etiquetas de clase comenzando con 0:

In [8]:
np.unique(df.clase)

array(['clase1', 'clase2'], dtype=object)

In [9]:
import numpy as np

# dictionario compreshion (it's better when there are many datos)
clase_map = {et: idx for idx, et in enumerate(np.unique(df.clase))}
clase_map

{'clase1': 0, 'clase2': 1}

A continuación podemos usar el diccionario para realizar la conversión de las etiquetas de la clase a enteros:

In [10]:
df['clase'] = df['clase'].map(clase_map)
df

,color,talla,precio,clase
0,verde,1,10.3,0
1,rojo,2,14.2,1
2,azul,3,15.6,0


Convenientemente existe la clase _LabelEncoder_ implementada en _scikit_learn_.

In [38]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

## Codificando en formato one-hot valores nominales

Podemos realizar un proceso similar par las columnas color (dado que no se requiere orden).

- http://blog.josemarianoalvarez.com/2018/03/15/categorias-y-la-codificacion-one-hot/

- https://ichi.pro/es/que-es-one-hot-encoding-y-como-usar-la-funcion-get-dummies-de-pandas-160729382340976

- https://machinelearningmastery.com/why-one-hot-encode-data-in-machine-learning/


In [ ]:
import numpy

In [11]:
color_map = [ [c, idx] for idx, c in enumerate(np.unique(df.color))]
color_map

[['azul', 0], ['rojo', 1], ['verde', 2]]

Si nos detuvieramos en este punto, estaríamos cayendo en uno de los errores más comúnes cuando se utilizan datos categóricos: los colores no tienen un orden particular; sim embargo un algoritmo de aprendizaje supondria que verde _es mayor que_ rojo y que rojo _es meyor que_ azul.
Esta suposición no es correcta y aunque se podría obtener un buen resultado, en general puede no ser óptimo.

Una posible solución es utilizar la técnica conocida como **codificacion _one hot_**. La idea básica es crear una nueva _característica ficticia_ (dummy) por cada valor único de la columna nominal correspondiente. En este caso, se generarán tres columnas: _azul, rojo y verde_. Se utilizan valores binarios para indicar el color particulas de cada muestra.

Para realizar esta transformación se puede utilizar el _OneHotEncoder_ disponible en el módulo _preprocessing_ de scikit-learn:

In [13]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(handle_unknown='ignore')
ohe.fit(color_map)
ohe.transform(color_map).toarray()

array([[1., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 1.]])

¿No deberían ser solamente tres columnas?

Por alguna razon aparecen seis, repitiendose las primeras tres nuevamente, lo importante son las primeras tres que indican si el color esta encendido(1) o apagado.

In [17]:
ohe.get_feature_names(['color', 'grupo'])


array(['color_azul', 'color_rojo', 'color_verde', 'grupo_0', 'grupo_1',
       'grupo_2'], dtype=object)

Una forma más conveniente para crear esas características ficticias para la codificación _one hot_, es utilizar el método _get_dummies_ implementado en pandas. Si se aplica a un _DataFrame_, este método convertirá las columnas de tipo _string_ y dejará las otras columnas sin cambios:

In [18]:
pd.get_dummies(df[['precio', 'color', 'talla']])

,precio,talla,color_azul,color_rojo,color_verde
0,10.3,1,0,0,1
1,14.2,2,0,1,0
2,15.6,3,1,0,0


Es importante tomar en cuenta que al utilizar este tipo de codificación se puede obtener problemas para algunos métodos(p.e, métodos que requieren calcular la matriz inversa). Si las características tienen correlación alta, las matrices son muy dificiles de invertir.

Para reducir la correlación de las variables, se puede simplemente eliminar una característica del arreglo _one hot_ obtenido, es importante notar que no se pierde información al eliminar una columna. El parámetro _drop_first_ de _get_dummies_ puede ser de ayuda:

In [19]:
pd.get_dummies(df[['precio', 'color', 'talla']],
              drop_first=True)

,precio,talla,color_rojo,color_verde
0,10.3,1,0,1
1,14.2,2,1,0
2,15.6,3,0,0
